In [ ]:
import csv
import pandas as pd
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn import metrics

import pickle
import math
import re
import enchant
import os
import glob
import numpy as np
np.random.seed(512)

from nltk.corpus import words
from dateutil.parser import parse
from datetime import datetime
import itertools

In [ ]:
def is_url(row, reg, pat):
    samples = row[12:17]
    for sample in samples:
        if reg.match(str(sample).strip()) and len(re.sub(pat, '', str(sample).strip())) == 0:
            return True
    return False


def is_datetime(row):
    samples = row[12:17]
    for sample in samples:
        try:
            _ = pd.Timestamp(sample)
            return True
        except ValueError:
            try:
                _ = pd.Timestamp(int(sample))
                return True
            except ValueError:
                continue
    return False


def is_email(row, reg):
    samples = row[12:17]
    for sample in samples:
        if reg.match(str(sample)):
            return True
    return False


def is_num(row, reg):
    samples = row[12:17]
    for sample in samples:
        try:
            _ = float(str(sample))
            return True
        except ValueError:
            if reg.match(str(sample).lower()):
                return True
    return False

url_pat = r"(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
url_reg = re.compile(url_pat)

list_pat =  r"((\d|\w|\')+(,|>|;|:|\-|`\.|\||\*){1}\s?(\d|\w|\')+){2,}"
list_reg = re.compile(list_pat)

num_pat = r"([\$|\w]?(\d|\d\.\d|\d\,\d)+\s?[\$|\w]?)"
num_reg = re.compile(num_pat)

email_pat = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,6}\b"
email_reg = re.compile(email_pat)

In [ ]:
testdf = pd.read_csv('../../data_test.csv')
test_metadata = pd.read_csv('../../RawCSV/Metadata/meta_data.csv')

test_merged = pd.merge(testdf,test_metadata,on='Record_id')
y_true = test_merged.y_act.values.tolist()

In [ ]:
def get_col_dtype(col):
        
    if col.dtype =="object":
        try:
            col_new = pd.to_numeric(col.dropna().unique())
            return 'Numeric'
        except:
            return "Object"
    if col.dtype =="bool": return "Object"
    else:
        if col.dtype == 'float64' or col.dtype == 'int64':
            return 'Numeric'
        else:      
            return col.dtype

In [ ]:
predictions = []
y_truth_iter = []
data = test_merged
prv_csv_name = ''

for index, data in test_merged.iterrows():
    if index%100==0: print(index)

    s1 = data['sample_1']
    avg_tokens = len(str(s1).split(' '))
    
    nans = data['%_nans']
    dist_val = data['%_dist_val']
    num_dist_val = data['num_of_dist_val']

    mean_wd_cnt = data['mean_word_count']
    std_wd_cnt = data['std_dev_word_count']
    has_delims =  data['has_delimiters']
    col = data['Attribute_name']
    
    csv_name = '../../RawCSV/RawCSVFiles/' + str(data['name'])
    
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')     
        
    try: df_col = df[col]
    except:
        predictions.append(1)
        continue    
    
    
    curtype = get_col_dtype(df_col)
    
    if str(curtype) == 'Numeric':
        y_truth_iter.append(data['y_act'])
        
        if nans > 99.99: 
            predictions.append(7)
            continue

        if num_dist_val < 10 :
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
        else:
            predictions.append(0)
            continue
    else:
        
        if nans > 99.99:
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
            continue

        if dist_val > 99.99:
            y_truth_iter.append(data['y_act'])
            predictions.append(7)
            continue
            
        if num_dist_val < 100 :
            y_truth_iter.append(data['y_act'])
            predictions.append(1)
        else:
            y_truth_iter.append(data['y_act'])
            
            if float(nans) >= 0.90:
                predictions.append(8)
            elif float(mean_wd_cnt) <= 2.0:
                if is_datetime(data):
                    predictions.append(2)
                elif is_email(data, email_reg):
                    predictions.append(8)
                elif is_url(data, url_reg, url_pat):
                    predictions.append(4)
                elif is_num(data, num_reg):
                    predictions.append(5)
                else:
                    predictions.append(8)
            elif float(std_wd_cnt) < 10.0:
                if has_delims == True:
                    predictions.append(6)
                else:
                    predictions.append(8)
            else:
                predictions.append(3)                
            continue
        

In [ ]:
print(accuracy_score(y_true, predictions))
print(confusion_matrix(y_true, predictions))